## June School Olympic Games

The students of the G. Galilei high school are enrolled in the June School Olympic Games. The instructors are planning the event schedule.

In a time span of 10 days, the students will be involved in matches of 100, 200, and 1000 meters, cross country running, discus throw, javelin throw, football, and baseball. The games will be played both in week days and week-end days.

The students have already enrolled for the sports they are interested in. Since some athlets will play matches of different disciplines, the activities must be schedules so as to entirely avoid overlaps.

The table reports, with an X, the pair of matches with at least one athlete in common. It also indicates, for each discipline, the total number of matches to be played.

Disciplines|100|200|1000|cross country|discus|javelin|football|baseball
-----------|---|---|----|-------------|------|-------|--------|--------
100|-|X|-|X|-|X|-|-
200|-|-|X|X|-|-|-|-
1000|-|-|-|X|-|-|X|X
cross country|-|-|-|-|-|X|-|-
discus|-|-|-|-|-|X|-|-
javelin|-|-|-|-|-|-|-|X
football|-|-|-|-|-|-|-|X
baseball|-|-|-|-|-|-|-|-
-----|-----|-----|---------------|-----|-----|-----|-----|-----
Total Matches|2|2|2|1|3|3|4|4


For each match, the instructors estimated the 'degree of interest', which is shown in the table

Disciplines|100|200|1000|cross country|discus|javelin|football|baseball
-----------|---|---|----|-------------|------|-------|--------|--------
Interest|7|8|10|4|5|6|10|9

The matches are played in sequence, during the first 3 afternoon hours (180 minutes). For each day, at most one match per discipline can be scheduled. The length, in minutes, of a match of each discipline is reported in the table

Disciplines|100|200|1000|cross country|discus|javelin|football|baseball
-----------|---|---|----|-------------|------|-------|--------|--------
Length|15|15|20|60|30|30|100|100

To exploit the enthusiasm of the athlets, the instructors decided that no more than 7 days must pass from the first and the last match of each discipline.

Give an integer linear programming formulation for the problem of determining a schedule for the matches such that the interest of the less interesting day is maximized. Per day, the interest amounts to the sum of the degree of interest of the matches that are played.

In [11]:
import mip
from mip import BINARY

In [12]:
# Types of matches
I = {'hundred', 'twohundred', 'thousand', 'crossCountry','discus', 'javelin', 'football', 'baseball'}

# Number of days
G = 10
# List of days
J = range(1, G+1)
# Available time per day
T = 180
# Maximum number of days between first and last match
n = 7

# Pair of matches with at least one athlete in common
c = [('hundred', 'twohundred'), ('hundred', 'crossCountry'), ('hundred', 'javelin'), \
     ('twohundred', 'thousand'), ('twohundred', 'crossCountry'), \
     ('thousand', 'crossCountry'), ('thousand', 'football'), ('thousand', 'baseball'), \
     ('crossCountry', 'javelin'), ('discus', 'javelin'), ('javelin', 'baseball'), ('football', 'baseball')]
# Total matches
m = {'hundred':2, 'twohundred':2, 'thousand':2, 'crossCountry':1,'discus':3, 'javelin':3, 'football':4, 'baseball':4}
# Degree of interest
w = {'hundred':7, 'twohundred':8, 'thousand':10, 'crossCountry':4,'discus':5, 'javelin':6, 'football':10, 'baseball':9}
# Length of a match of each discipline
t = {'hundred':15, 'twohundred':15, 'thousand':20, 'crossCountry':60, 'discus':30, 'javelin':30, 'football':100, 'baseball':100}

In [13]:
model = mip.Model()

In [21]:
x = {(i, str(j)): model.add_var(var_type=BINARY, name = i+'_'+str(j)) for i in I for j in J}
eta = model.add_var(lb=0, name = 'eta')

In [22]:
model.objective = mip.maximize(eta)

In [23]:
# Interest constraint
for j in J:
    model.add_constr(mip.xsum(w[i]*x[i, str(j)] for i in I) >= eta)

# Conflicts constraint
for j in J:
    for i in I:
        for i1 in I:
            if (i, i1) in c:
                model.add_constr(x[i, str(j)]+x[i1, str(j)] <= 1)

# Number constraint
for i in I:
    model.add_constr(mip.xsum(x[i, str(j)] for j in J) == m[i])

# Time constraint
for j in J:
    model.add_constr(mip.xsum(t[i]*x[i, str(j)] for i in I) <= T)

# Limit constraint
for i in I:
    for j in J:
        for j1 in J:
            if (j - j1) > n:
                model.add_constr(x[i, str(j)]+x[i, str(j1)] <= 1)

In [24]:
# optimizing
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [25]:
model.objective.x

14.0

In [26]:
for i in model.vars:
    if i.x != 0:
        print(i.name, i.x)

discus_6 1.0
discus_7 1.0
discus_10 1.0
hundred_7 1.0
hundred_10 1.0
crossCountry_1 1.0
javelin_3 1.0
javelin_4 1.0
javelin_5 1.0
baseball_2 1.0
baseball_6 1.0
baseball_8 1.0
baseball_9 1.0
football_1 1.0
football_3 1.0
football_4 1.0
football_5 1.0
twohundred_8 1.0
twohundred_9 1.0
thousand_7 1.0
thousand_10 1.0
discus_1 1.0
discus_4 1.0
discus_5 1.0
hundred_7 1.0
hundred_10 1.0
crossCountry_6 1.0
javelin_2 1.0
javelin_3 1.0
javelin_9 1.0
baseball_1 1.0
baseball_4 1.0
baseball_7 1.0
baseball_8 1.0
football_2 1.0
football_3 1.0
football_6 1.0
football_9 1.0
twohundred_1 1.0
twohundred_8 1.0
thousand_5 1.0
thousand_10 1.0
eta 14.0


<h3 align="center">Formulation</h3> 
 
- Sets
    - $I$: disciplines
    - $J= \{1,...,G\}$: days
    
- Parameters
    - T: available time per day
    - n: maximum number of days between first and last matc
    - $c_{ii'}$: 1 if $i$-th and $i'$-th matches are in conflict $i,i' \in I$
    - $m_i$: number of matches for discipline $i \in I$
    - $w_i$: degree of interest for discipline $i \in I$
    - $t_i$: length of a match for discipline $i \in I$

- Decision variables
    - $x_{ij}$: 1 if a match of discipline $i$ is played in day $j,i\in I, j \in J$
    - $\eta$: interest of the less interesting day
    
- Model
  $$
\begin{array}{lll}
  \max & \eta \qquad & & \text{(min interest)}\\
  \textrm{s. t.}
  & \sum_{i \in I} x_{ij} w_i \geq \eta &  j \in J & \text{(interest)} &\\
  & x_{ij}+x_{i^*j} \leq 1 &  i,i^* \in I,j \in J: c_{ii^*}=1 & \text{(conflicts)} &\\
  & \sum_{j \in J} x_{ij}=m_i  &  i \in I & \text{(number)} &\\
  & \sum_{j \in J}t_i x_{ij}\leq T  &  j \in J & \text{(time)} &\\
  & x_{ij}+x_{ij^*} \leq 1 &  i \in I,j,j^* \in J: j - j^* > n & \text{(limit)} &\\
  & x_{ij} \in \{0,1\} &  i\in I,j \in J & \text{(binary variables)} &\\
  & \eta \geq 0 & & \text{(nonnegative variables)} &\\
\end{array}
 $$
